In [1]:
import multiprocessing
from pyseqrna import pyseqrna_stats as ps
from pyseqrna import pyseqrna_utils as pu
from pyseqrna import quality_check as qc
from pyseqrna import quality_trimming as qt
from pyseqrna import  aligners as al
from pyseqrna import quantification as quants
from pyseqrna import differential_expression as de
from pyseqrna import pyseqrna_plots as pp
from pyseqrna import multimapped_groups as mmg
import pandas as pd
import dill

In [2]:
data = pu.read_input_file("/home/naveen/Downloads/pyseqrna_test/input_sample.txt", "/home/naveen/Downloads/pyseqrna_test/" , paired=True)
samples= data['samples']

[17:53:59]  pyseqrna_utils :: INFO : Reading input samples File 
[17:53:59]  pyseqrna_utils :: INFO : Input file /home/naveen/Downloads/pyseqrna_test/input_sample.txt read succesfully
[17:53:59]  pyseqrna_utils :: INFO : Combination created succesfully from /home/naveen/Downloads/pyseqrna_test/input_sample.txt
[17:53:59]  pyseqrna_utils :: INFO : targets dataframe for differenatial created succesfully from /home/naveen/Downloads/pyseqrna_test/input_sample.txt


In [4]:
processes = []
manager = multiprocessing.Manager()
rdict = manager.dict()
for sp in samples:
    p=multiprocessing.Process(target=ps.getNreads, args=(samples[sp][2],rdict, sp))
    
    processes.append(p)
    p.start()
        
for process in processes:
    process.join()
print(rdict)


{'WT3': 15901761, 'F2': 17952531, 'PF1': 16767110, 'PF3': 15928550, 'F3': 16633124, 'PF2': 16761019, 'WT2': 18418491, 'F1': 18873923, 'WT1': 19820972}
